### Dataset 과 DataLoader

#### Dataset

머신러닝, 딥러닝 학습에 사용되는 방대한 데이터의 크기 때문에 데이터를 한 번에 불러오기 쉽지 않습니다. 따라서 데이터를 한 번에 부르지 않고 하나씩만 불러서 쓰는 방식을 택해야 합니다. 모든 데이터를 불러놓고 사용하는 기존의 Dataset 말고 Custom Dataset 이 필요합니다.



In [ ]:
# 기본 템플릿
class CustomDataset(Dataset):
    def __init__(self):
        # 생성자, 데이터를 전처리 하는 부분
        pass

    def __len__(self):
        # 데이터셋의 총 길이를 반환하는 부분
        pass

    def __getitem__(self,idx):
        # idx(인덱스)에 해당하는 입출력 데이터를 반환
        pass

##### 선형 회귀를 위해 Dataset

In [3]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                            [93, 99, 93]]
        self.y_data = [[152], [185]]

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

dataset = CustomDataset()

#### DataLoader

- DataLoader는 PyTorch 데이터 로딩 유틸리티의 핵심입니다. DataLoader의 가장 중요한 인자는 데이터를 불러올 데이터셋 객체를 나타내는 데이터셋입니다.
- DataLoader는 iterator 형식으로 데이터에 접근 하도록 하며 batch_size나 shuffle 유무를 설정

In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size = 2,
    shuffle = True,
)


##### 데이터로더 정의

In [6]:
DataLoader(dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=None,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

#### collate_fn

DataLoader에는 collate_fn 이라는 파라미터를 지정할 수가 있습니다. 이 파라미터를 사용하면 별도의 데이터 처리 함수를 만들 수 있으며 해당 함수 내의 처리를 데이터가 출력되기 전에 적용됩니다. 기본적으로 default_collate라는 함수는 Dataset이 반환하는 데이터 유형을 확인하고 (x_batch, y_batch) 와 같은 배치로 결합하려고 시도합니다 

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        """
        data: list of tuples, e.g., [(text1, label1), (text2, label2), ...]
        """
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        return text, label


# 예시 데이터
data = [
    ([1., 0., 45.], 1),
    ([2., 1., 50.], 0),
    ([0., 3., 30.], 1),
]

# TD는 MyDataset 클래스의 인스턴스
TD = MyDataset(data)

In [10]:
import torch

def collate_batch(batch):
    word_tensor = torch.tensor([[1.], [0.], [45.]])
    label_tensor = torch.tensor([[1.]])

    text_list, classes = [], []
    for (_text, _class) in batch:
        text_list.append(word_tensor)
        classes.append(label_tensor)
        
    text = torch.cat(text_list)
    classes = torch.tensor(classes)
    return text, classes
DL_DS = DataLoader(TD, batch_size=2, collate_fn=collate_batch)

#### Transform

Pytorch는 이미지 분류, segmentation, 텍스트 처리, object Identification과 같은 다양한 작업에 광범위하게 사용되는 딥 러닝 프레임워크입니다. 이러한 경우 다양한 유형의 데이터를 처리해야 합니다. 그리고 대부분의 경우 데이터가 데이터가 항상 머신러닝 알고리즘 학습에 필요한 최종 처리가 된 형태로 제공되지는 않습니다. transform 을 해서 데이터를 조작하고 학습에 적합하게 만들어야 합니다.

- PyTorch의 torchvision 라이브러리는 transforms 에서 다양한 변환 기능을 제공합니다. transform을 사용하여 데이터의 일부 조작을 수행하고 훈련에 적합하게 만들 수 있습니다.

    - transforms.ToTensor() - 데이터를 tensor로 바꿔준다.
    - transforms.Normalize(mean, std, inplace=False) - 정규화
    - transforms.ToPILImage() - csv 파일로 데이터셋을 받을 경우, PIL image로 바꿔준다.
    - transforms.Compose - 여러 단계로 변환해야 하는 경우, Compose를 통해 여러 단계를 묶을 수 있다.

Dataset 클래스의 `__getitem__` 함수내에서 데이터를 변환하여 리턴될 때 주로 사용됩니다.

In [ ]:
from torchvision import transforms, utils
import os

class TorchvisionMaskDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.imgs = list(sorted(os.listdir(self.path)))
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        file_image = self.imgs[idx]
        file_label = self.imgs[idx][:-3] + 'xml'
        img_path = os.path.join(self.path, file_image)
        # ....
        #....
        target = None

        if self.transform:
            img = self.transform(img)

        return img, target

torchvision_transform = transforms.Compose([
    transforms.Resize((300, 300)), 
    transforms.RandomCrop(224),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomHorizontalFlip(p = 1),
    transforms.ToTensor(),
])

torchvision_dataset = TorchvisionMaskDataset(
    path = './images/',
    transform = torchvision_transform
)

1. ToTensor
    - ToTensor는 매우 일반적으로 사용되는 conversion transform입니다.
    - `transforms.ToTensor()`

2. Normalize
    - Normalize 작업은 텐서를 가져와 평균 및 표준 편차로 정규화

3. CenterCrop
    - 이것은 중앙에서 주어진 텐서 이미지를 자릅니다. `transform.CenterCrop(height, width)` 형식

4. RandomHorizontalFlip
    - 이 변환은 주어진 확률로 이미지를 무작위로 수평으로 뒤집을(flip) 것입니다.

5. RandomRotation
    - 이 변환은 이미지를 각도만큼 무작위로 회전합니다.

6. Grayscale
    - 이 변환은 원본 RGB 이미지를 회색조로 변경합니다. 

7. 가우시안 블러
    - 여기에서 이미지는 무작위로 선택된 가우시안 흐림 효과로 흐려집니다. 

8. RandomApply
    - 이 변환은 확률로 주어진 transformation 들을 무작위로 적용합니다.

In [ ]:
transform = transforms.RandomApply([transforms.RandomSizedCrop(200),transforms.RandomHorizontalFlip()],p=0.6)
tensor_img = transform(image)

9. Compose
    - transform에 여러 단계가 있는 경우, Compose를 통해 여러 단계를 하나로 묶을 수 있습니다.

In [ ]:
transforms.Compose([ 
   transforms.ToTensor(), 
   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])
